# Tune Hyperparameters

There are many machine learning algorithms that require *hyperparameters* (parameter values that influence training, but can't be determined from the training data itself). For example, when training a logistic regression model, you can use a *regularization rate* hyperparameter to counteract bias in the model; or when training a convolutional neural network, you can use hyperparameters like *learning rate* and *batch size* to control how weights are adjusted and how many data items are processed in a mini-batch respectively. The choice of hyperparameter values can significantly affect the performance of a trained model, or the time taken to train it; and often you need to try multiple combinations to find the optimal solution.

In this case, you'll train a classification model with two hyperparameters, but the principles apply to any kind of model you can train with Azure Machine Learning.

## Connect to your workspace

To get started, connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [47]:
import azureml.core
from azureml.core import Workspace

import logging
import os
import csv
from datetime import datetime
import pytz


from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import seaborn as sns
import tensorflow as tf

import matplotlib.pyplot as plt
import re
import pydot
import graphviz

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
# print("SDK version:", azureml.core.VERSION)

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

print("Tensorflow version: ", tf.__version__)
print("Current DateTime: ", datetime.now(pytz.timezone("America/New_York")).strftime("%m/%d/%Y %H:%M:%S"))

Ready to use Azure ML 1.41.0 to work with nahmed30-azureml-workspace
nahmed30-azureml-workspace
epe-poc-nazeer
centralus
16bc73b5-82be-47f2-b5ab-f2373344794c
Tensorflow version:  2.2.0
Current DateTime:  08/28/2022 00:18:00


In [50]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

## Prepare data

In this lab, you'll use a dataset containing details of diabetes patients. Run the cell below to create this dataset (if it already exists, the existing version will be used)

In [51]:
from azureml.core import Dataset

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "UdacityPrjEmailSpamDataSet"
description_text = "Spam Detection DataSet for Udacity Capstone Proj "

dataset = None
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 



df = dataset.to_pandas_dataframe()
df.describe()


,v1,v2,Column3,Column4,Column5
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""",GE,"GNT:-)"""
freq,4825,30,3,2,2


## Prepare a training script

Now let's create a folder for the training script you'll use to train the model.

In [52]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_folder = 'spam_training-hyperdrive'
experiment_name = 'ml-spam-experiment-prjassign1'
project_folder = 'spam_training-hyperdrive'

os.makedirs(project_folder, exist_ok=True)

print('Project Folder is ready.')

experiment = Experiment(ws, experiment_name)
experiment

Project Folder is ready.


Name,Workspace,Report Page,Docs Page
ml-spam-experiment-prjassign1,nahmed30-azureml-workspace,Link to Azure Machine Learning studio,Link to Documentation


## Create compute

Hyperparameter tuning involves running multiple training iterations with different hyperparameter values and comparing the performance metrics of the resulting models. To do this efficiently, we'll take advantage of on-demand cloud compute and create a cluster - this will allow multiple training iterations to be run concurrently.

Use the following code to specify an Azure Machine Learning compute cluster (it will be created if it doesn't already exist).

> **Important**: Change *your-compute-cluster* to the name of your compute cluster in the code below before running it! Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.

In [53]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


> **Note**: Compute instances and clusters are based on standard Azure virtual machine images. For this exercise, the *Standard_DS11_v2* image is recommended to achieve the optimal balance of cost and performance. If your subscription has a quota that does not include this image, choose an alternative image; but bear in mind that a larger image may incur higher cost and a smaller image may not be sufficient to complete the tasks. Alternatively, ask your Azure administrator to extend your quota.

You'll need a Python environment to be hosted on the compute, so let's define that as Conda configuration file.

In [ ]:
%%writefile $project_folder/hyperdrive_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip
- pip:
  - azureml-defaults


In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(18, 8))
plt.subplot(1, 2, 1)
df.v1.value_counts().plot(kind="pie",
                                           fontsize=16,
                                           labels=["Spam", "Ham"],
                                           ylabel="Spam vs Ham",
                                           autopct='%1.1f%%');

plt.subplot(1, 2, 2)
sns.countplot(x="v1",data=df, palette="pastel")
plt.show()

In [ ]:
plt.figure(figsize=(18, 8))
plt.subplot(1, 2, 1)
ax1 = df.query("v1=='spam'").v2.map(lambda x: len(x.split())).plot(kind="hist",
                                                                    color="cyan",
                                                                    title="Spam",
                                                                    edgecolor='white');
plt.subplot(1, 2, 2)
ax2 = df.query("v1=='ham'").v2.map(lambda x: len(x.split())).plot(kind="hist",
                                                                    color="orange",
                                                                    title="Ham",
                                                                    edgecolor='white');

ax1.grid(visible = True, color ='grey',linestyle ='-.', linewidth = 0.5,alpha = 0.6)
ax2.grid(visible = True, color ='grey',linestyle ='-.', linewidth = 0.5,alpha = 0.6)
plt.suptitle('Word distribution in SMS')
plt.show()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from wordcloud import WordCloud
stop_words= set(stopwords.words("english"))

stop_words.update(['https', 'http', 'amp', 'CO', 't', 'u', 'new', "I'm", "would"])

wc = WordCloud(width=800,
               height=400,
               max_words=200,
               stopwords=stop_words,
               background_color='white',
               max_font_size=150)
spam = df.query("v1=='spam'").v2.str.cat(sep=" ")

ham = df.query("v1=='ham'").v2.str.cat(sep=" ")

print('\n\nWord Cloud for Spam messages\n\n')
wc.generate(spam)
plt.figure(figsize=(16, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

print('\n\nWord Cloud for Ham messages\n\n')
wc.generate(ham)
plt.figure(figsize=(16, 8))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
df = df.replace('spam', 1)
df = df.replace('ham', 0)
df

In [ ]:
def cleanText(text):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = text.replace('.', '')
    text = whitespace.sub(' ', text)
    text = web_address.sub('', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    return text.lower()

df.v2 = [cleanText(item) for item in df.v2]
df

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<oovToken>'
tokenizer.fit_on_texts(df.v2)
vocab = tokenizer.word_index
vocabCount = len(vocab)+1

vocabCount

In [ ]:
SPLIT = 5000

xTrain = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(df.v2.to_numpy()), padding='pre', maxlen=171)
yTrain = df.v1.to_numpy()
dim = xTrain.shape[1]
xTest = xTrain[SPLIT:]
yTest = yTrain[SPLIT:]

xTrain = xTrain[:SPLIT]
yTrain = yTrain[:SPLIT]

xTrain.shape, yTrain.shape, xTest.shape, yTest.shape

**Building and Fitting the models**

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocabCount+1, output_dim=64, input_length=dim))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(xTrain, yTrain, epochs=10, shuffle=True)

In [ ]:
model.evaluate(xTest, yTest)

In [ ]:
text = "Congratulations, you have won a 10000 dollars lottery. Please give your bank details to claim the money"
processedText = cleanText(text)
#print(processedText)
finalText = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([processedText]), padding='pre', maxlen=171)
prediction = model.predict(finalText)
print("prediction shape", prediction.shape)
print(prediction)

print(np.int(np.rint(prediction[0,0])))

In [ ]:
text = "I'll meet you at nariman point tomorrow"
processedText = cleanText(text)
#print(processedText)
finalText = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([processedText]), padding='pre', maxlen=171)
prediction = model.predict(finalText)
print("prediction shape", prediction.shape)
print(prediction)
print(np.int(np.rint(prediction[0,0])))

In [ ]:
text = "You won $100 click link below to collect"
processedText = cleanText(text)
#print(processedText)
finalText = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([processedText]), padding='pre', maxlen=171)
prediction = model.predict(finalText)
print("prediction shape", prediction.shape)
print(prediction)
print(np.int(np.rint(prediction[0,0])))

## Run a hyperparameter tuning experiment

Azure Machine Learning includes a hyperparameter tuning capability through *hyperdrive* experiments. These experiments launch multiple child runs, each with a different hyperparameter combination. The run producing the best model (as determined by the logged target performance metric for which you want to optimize) can be identified, and its trained model selected for registration and deployment.

> **Note**: In this example, we aren't specifying an early stopping policy. Such a policy is only relevant if the training script performs multiple training iterations, logging the primary metric for each iteration. This approach is typically employed when training deep neural network models over multiple *epochs*.

https://www.tensorflow.org/tutorials/keras/keras_tuner

In [ ]:
import keras_tuner as kt

In [55]:
def build_model (hp):
    hpmodel = tf.keras.Sequential()
    hpmodel.add(tf.keras.layers.Embedding(input_dim=vocabCount+1, output_dim=64, input_length=dim))
    hpmodel.add(tf.keras.layers.GlobalAveragePooling1D())
    for i in range(hp.Int('num_layers', 2, 20)):
        hpmodel.add(layers.Dense(units=hp.Int('units_' + str(i),
                                                min_value=32,
                                                max_value=512,
                                                step=32),
                                 activation='relu'))
    hpmodel.add(tf.keras.layers.Dense(1, activation='sigmoid'))    
    hpmodel.compile(loss='binary_crossentropy', 
                    optimizer=tf.keras.optimizers.Adam(
                        hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                    metrics=['accuracy'])
    return hpmodel

## ------------------------------------------

#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Embedding(input_dim=vocabCount+1, output_dim=64, input_length=dim))
#model.add(tf.keras.layers.GlobalAveragePooling1D())
#model.add(tf.keras.layers.Dense(64, activation='relu'))
#model.add(tf.keras.layers.Dense(32, activation='relu'))
#model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



def model_builder(hp):
  modelhp = tf.keras.Sequential()

  modelhp.add(tf.keras.layers.Embedding(input_dim=vocabCount+1, output_dim=64, input_length=dim))
  modelhp.add(tf.keras.layers.GlobalAveragePooling1D())
  modelhp.add(tf.keras.layers.Dense(64, activation='relu'))
  modelhp.add(tf.keras.layers.Dense(32, activation='relu'))
  modelhp.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  modelhp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # modelhp.add(tf.keras.layers.Flatten(input_shape=(32, 171)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  modelhp.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  modelhp.add(tf.keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  modelhp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return modelhp

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='spam_training-hyperdrive',
                     project_name='hyperparameter_kt')

In [56]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='spam_training-hyperdrive',
    project_name='hyperparameter_kt'
)

INFO:tensorflow:Reloading Oracle from existing project spam_training-hyperdrive/hyperparameter_kt/oracle.json
INFO:tensorflow:Reloading Tuner from spam_training-hyperdrive/hyperparameter_kt/tuner0.json


In [57]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [64]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [67]:
tuner.search(xTrain, yTrain, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]



INFO:tensorflow:Oracle triggered exit


<bound method HyperParameters.get_config of <keras_tuner.engine.hyperparameters.HyperParameters object at 0x7fe333f91220>>

In [68]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.01.



In [70]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
modelhp = tuner.hypermodel.build(best_hps)
history = modelhp.fit(xTrain, yTrain, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
125/125 [==============================] - 0s 4ms/step - loss: 0.2519 - accuracy: 0.9075 - val_loss: 0.1328 - val_accuracy: 0.9590
Epoch 2/50
125/125 [==============================] - 0s 3ms/step - loss: 0.0567 - accuracy: 0.9835 - val_loss: 0.0563 - val_accuracy: 0.9830
Epoch 3/50
125/125 [==============================] - 0s 3ms/step - loss: 0.0275 - accuracy: 0.9915 - val_loss: 0.0650 - val_accuracy: 0.9830
Epoch 4/50
125/125 [==============================] - 0s 3ms/step - loss: 0.0204 - accuracy: 0.9942 - val_loss: 0.0705 - val_accuracy: 0.9850
Epoch 5/50
125/125 [==============================] - 0s 3ms/step - loss: 0.0100 - accuracy: 0.9970 - val_loss: 0.1019 - val_accuracy: 0.9830
Epoch 6/50
125/125 [==============================] - 0s 4ms/step - loss: 0.0153 - accuracy: 0.9960 

hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(xTrain, yTrain, epochs=best_epoch, validation_split=0.2)


In [71]:
eval_result = modelhp.evaluate(xTest, yTest)
print("[test loss, test accuracy]:", eval_result)

18/18 [==============================] - 0s 927us/step - loss: 0.1309 - accuracy: 0.9860
[test loss, test accuracy]: [0.13093294203281403, 0.9860140085220337]


In [72]:
text = "You won $100 click link below to collect"
processedText = cleanText(text)
#print(processedText)
finalText = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([processedText]), padding='pre', maxlen=171)
prediction = modelhp.predict(finalText)
print("prediction shape", prediction.shape)
print(prediction)
print(prediction[0,0])
print(np.int(np.rint(prediction[0,0])))

prediction shape (1, 1)
[[0.9999424]]
0.9999424
1


In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
hyper_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/hyperdrive_env.yml")

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=hyper_env,
                                compute_target = training_cluster)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# Run the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

In [73]:
text = "I'll meet you at nariman point tomorrow"
processedText = cleanText(text)
#print(processedText)
finalText = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([processedText]), padding='pre', maxlen=171)
prediction = modelhp.predict(finalText)
print("prediction shape", prediction.shape)
print(prediction)
print(np.int(np.rint(prediction[0,0])))

prediction shape (1, 1)
[[1.3286123e-09]]
0


You can view the experiment run status in the widget above. You can also view the main Hyperdrive experiment run and its child runs in [Azure Machine Learning studio](https://ml.azure.com).

> **Note**: If a message indicating that a non-numeric can't be visualized is displayed, you can ignore it.

## Determine the best performing run

When all of the runs have finished, you can find the best one based on the performance metric you specified (in this case, the one with the best AUC).

In [ ]:
# Print all child runs, sorted by the primary metric
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

Now that you've found the best run, you can register the model it trained.

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **More Information**: For more information about Hyperdrive, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters).